In [3]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', 5000)
import warnings
warnings.filterwarnings(action='ignore')

from sklearn.model_selection import train_test_split

In [4]:
from pathlib import Path

ROOT_DIR = Path('../')
DATA_DIR = ROOT_DIR / 'data'
MODELS_DIR = ROOT_DIR / 'models'

train_filepath = DATA_DIR / 'loan_eligibility.xlsx'
test_filepath = DATA_DIR / 'loan_eligibility.xlsx'

In [5]:
import sys
sys.path.append('../')
sys.path.append('../pipelines')
sys.path

['C:\\Users\\wakar\\loan_prediction_production-master\\notebooks',
 'C:\\ProgramData\\Anaconda3\\python38.zip',
 'C:\\ProgramData\\Anaconda3\\DLLs',
 'C:\\ProgramData\\Anaconda3\\lib',
 'C:\\ProgramData\\Anaconda3',
 '',
 'C:\\Users\\wakar\\AppData\\Roaming\\Python\\Python38\\site-packages',
 'C:\\ProgramData\\Anaconda3\\lib\\site-packages',
 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\locket-0.2.1-py3.8.egg',
 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\wakar\\.ipython',
 '../',
 '../pipelines']

In [9]:
from pipelines.inference import make_dag_predictions

In [6]:
df_raw = pd.read_excel('../data/loan_eligibility.xlsx')

In [7]:
df = df_raw.sample(n=1000)

In [8]:
df_dict = df.to_dict(orient='records')

In [16]:
df_dict

[{'Loan_ID': '5f932165-a5e4-4096-8fa6-be0479eaa823',
  'Customer_ID': '06cecac6-cdb5-4abc-ad4f-7d30b72dd336',
  'Loan_Status': 'Charged Off',
  'Current_Loan_Amount': 540892,
  'Term': 'Short Term',
  'Credit_Score': 739.0,
  'Annual_Income': 1557088.0,
  'Years_in_current_job': '10+ years',
  'Home_Ownership': 'Own Home',
  'Purpose': 'Debt Consolidation',
  'Monthly_Debt': 20112.26,
  'Years_of_Credit_History': 18.7,
  'Months_since_last_delinquent': nan,
  'Number_of_Open_Accounts': 19,
  'Number_of_Credit_Problems': 0,
  'Current_Credit_Balance': 662568,
  'Maximum_Open_Credit': 2067868.0,
  'Bankruptcies': 0.0,
  'Tax_Liens': 0.0},
 {'Loan_ID': 'a53a0e22-28ec-425d-ae62-2c82e3adf00c',
  'Customer_ID': '6d30e7b0-1af8-4e36-ad6c-1835605ac667',
  'Loan_Status': 'Fully Paid',
  'Current_Loan_Amount': 370744,
  'Term': 'Short Term',
  'Credit_Score': 711.0,
  'Annual_Income': 726180.0,
  'Years_in_current_job': '10+ years',
  'Home_Ownership': 'Rent',
  'Purpose': 'Debt Consolidation',
 

In [14]:
import json
json = json.dumps(df_dict[0])

In [15]:
json

'{"Loan_ID": "5f932165-a5e4-4096-8fa6-be0479eaa823", "Customer_ID": "06cecac6-cdb5-4abc-ad4f-7d30b72dd336", "Loan_Status": "Charged Off", "Current_Loan_Amount": 540892, "Term": "Short Term", "Credit_Score": 739.0, "Annual_Income": 1557088.0, "Years_in_current_job": "10+ years", "Home_Ownership": "Own Home", "Purpose": "Debt Consolidation", "Monthly_Debt": 20112.26, "Years_of_Credit_History": 18.7, "Months_since_last_delinquent": NaN, "Number_of_Open_Accounts": 19, "Number_of_Credit_Problems": 0, "Current_Credit_Balance": 662568, "Maximum_Open_Credit": 2067868.0, "Bankruptcies": 0.0, "Tax_Liens": 0.0}'

In [13]:
predictions = make_dag_predictions(df, MODELS_DIR)

In [17]:
a = pd.DataFrame(data=predictions, columns = ["predictions"])

In [18]:
a

,predictions
0,1
1,1
2,1
3,1
4,1
...,...
995,0
996,1
997,1
998,1


In [6]:
url = 'http://127.0.0.1:8000/dag_predictions'
import requests

df = df_raw.sample(n=1000)
def test(df):
    #X = pd.read_excel("../data/loan_eligibility.xlsx")
    df_json = df_raw.to_json(orient='records', date_format='iso') 
    predictions = requests.post(url, json=df_json)
        
    return predictions


print(test(df))

<Response [422]>


In [7]:
url = 'http://127.0.0.1:8000/dag_predictions'
import requests
def test1(path):
    df = requests.post(url, data=path)
    return df
    
print(test1('../data/loan_eligibility.xlsx'))

<Response [422]>


In [3]:
train = df_raw.copy()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Loan_ID                       100000 non-null  object 
 1   Customer_ID                   100000 non-null  object 
 2   Loan_Status                   100000 non-null  object 
 3   Current_Loan_Amount           100000 non-null  int64  
 4   Term                          100000 non-null  object 
 5   Credit_Score                  80846 non-null   float64
 6   Annual_Income                 80846 non-null   float64
 7   Years_in_current_job          95778 non-null   object 
 8   Home_Ownership                100000 non-null  object 
 9   Purpose                       100000 non-null  object 
 10  Monthly_Debt                  100000 non-null  float64
 11  Years_of_Credit_History       100000 non-null  float64
 12  Months_since_last_delinquent  46859 non-null 

In [4]:
X = train
y = X.pop('Loan_Status')

### Preprocessing

In [5]:
def drop_cols_with_missing_vals(df, percentage = 50):
    percent_missing = train.isnull().sum() * 100 / len(train)
    missing_value_df = pd.DataFrame({'column_name': train.columns, 'percent_missing': percent_missing})
    cols_to_drop = missing_value_df[missing_value_df['percent_missing'] > percentage]
    cols_to_drop = cols_to_drop['column_name'].tolist()
    
    return df.drop(cols_to_drop, axis=1)

In [6]:
X = drop_cols_with_missing_vals(X, 50)
X = X.drop(['Loan ID', 'Customer ID'], axis=1)

KeyError: "['Loan ID' 'Customer ID'] not found in axis"

In [ ]:
num_cols = X._get_numeric_data().columns.to_list()
cat_cols = X.select_dtypes('object').columns

#### Categorical Missing Values

In [ ]:
X.head()

In [ ]:
for col in cat_cols:
    X[col] = X[col].fillna(X[col].mode()[0])

#### Numerical Missing Values

In [ ]:
for col in num_cols:
    X[col] = X[col].fillna(X[col].mean())

#### Encoding Categoricals

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

oh_encoder = ColumnTransformer(transformers=[('encoder', OneHotEncoder(handle_unknown = 'ignore'), cat_cols)], remainder='passthrough')
X = oh_encoder.fit_transform(X)


In [ ]:
y = y.replace({'Fully Paid': 1, 'Charged Off': 0})

### Model Training

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

In [ ]:
data = {
    "Current_Loan_Amount": 450000,
    "Term": "Short Term",
    "Credit_Score": 700,
    "Annual_Income": 100000,
    "Years_in_current_job": "8 years",
    "Home_Ownership": "Home Mortgage",
    "Purpose": "Home Improvements",
    "Monthly_Debt": 5200.25,
    "Years_of_Credit_History": 17.2,
    "Months_since_last_delinquent": 5,
    "Number_of_Open_Accounts": 6,
    "Number_of_Credit_Problems": 1,
    "Current_Credit_Balance": 228100,
    "Maximum_Open_Credit": 416746,
    "Bankruptcies": 1,
    "Tax_Liens": 0
}

In [ ]:
data

In [ ]:
#y_pred = classifier.predict([445412,Short Term,709.0,1167493.0,8,years,Home Mortgage,Home Improvements,5214.74,17.2,6,1,228190,416746.0,1.0,0.0])
y_pred = classifier('C:/Users/Souran/Downloads/dsp-waqar-alvi/dsp-waqar-alvi/models/model_reg_multi.joblib')

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix, classification_report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
print('Recall Score for testing data is',recall_score(y_test,y_pred))
print('Precision Score for testing data is',precision_score(y_test,y_pred))
print('F1 Score for testing data is',f1_score(y_test,y_pred))
print('Accuracy Score for testing data is',accuracy_score(y_test,y_pred))